# compare runs, choose a model, and deploy using REST API

In [2]:
import numpy as np
import pandas as pd
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorflow as tf
import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, TensorSpec

# Disable GPU if not available
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Load the dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data
X, y = data.iloc[:, :-1], data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check for NaN values in the data
print("NaN values in X_train:", np.isnan(X_train).sum())
print("NaN values in X_test:", np.isnan(X_test).sum())
print("NaN values in y_train:", np.isnan(y_train).sum())
print("NaN values in y_test:", np.isnan(y_test).sum())

# Handle NaN values if any
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

def train_model(X_train, y_train, X_test, y_test, params):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(params["units"], activation=params["activation"]),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=params["optimizer"], loss="mse")
    
    # Debugging: Print hyperparameters
    print("Training with params:", params)
    
    history = model.fit(X_train, y_train, epochs=params["epochs"], verbose=0)
    
    # Debugging: Print training loss
    print("Training loss:", history.history["loss"])
    
    mse = model.evaluate(X_test, y_test, verbose=0)
    
    # Debugging: Print test loss
    print("Test loss (MSE):", mse)
    
    # Log the model with MLflow
    input_schema = Schema([TensorSpec(np.dtype(np.float64), (-1, X_train.shape[1]))])
    output_schema = Schema([TensorSpec(np.dtype(np.float64), (-1, 1))])
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)
    mlflow.tensorflow.log_model(model, artifact_path="model", signature=signature)
    
    # Log metrics
    mlflow.log_metric("mse", mse)
    
    return {"loss": mse, "status": STATUS_OK, "model": model, "signature": signature}

def objective(params):
    params["epochs"] = 3
    result = train_model(
        X_train, 
        y_train, 
        X_test, 
        y_test, 
        params
    )
    if np.isnan(result["loss"]):
        result["loss"] = 1e6  # Penalize invalid hyperparameter combinations
    return result

space = {
    "units": hp.choice("units", [32, 64, 128, 256, 512]),
    "activation": hp.choice("activation", ["relu", "tanh", "sigmoid"]),
    "output_activation": hp.choice("output_activation", ["relu", "tanh", "sigmoid"]),
    "optimizer": hp.choice("optimizer", ["SGD", "adam"]),
    "learning_rate": hp.uniform("learning_rate", 0.0001, 0.01),  # Reduced upper bound
    "momentum": hp.uniform("momentum", 0.1, 0.9),
    "loss": hp.choice("loss", ["mean_squared_error"]),
    "batch_size": hp.choice("batch_size", [32, 64, 128]),
    "epochs": 3
}

mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=3,
        trials=trials
    )
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best params, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("mse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=best_run["signature"])

    print(f"Best params: {best}")
    print(f"Best MSE: {best_run['loss']}")

# End the run after completion
mlflow.end_run()

NaN values in X_train: fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
dtype: int64
NaN values in X_test: fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
dtype: int64
NaN values in y_train: 0
NaN values in y_test: 0
Training with params:                                
{'activation': 'sigmoid', 'batch_size': 64, 'epochs': 3, 'learning_rate': 0.0013480015876999667, 'loss': 'mean_squared_error', 'momentum': 0.4054876249102949, 'optimizer': 'SGD', 'output_activation': 'sigmoid', 'units': 32}
Training loss:                

In [3]:
# best hyperparameters

In [ ]:
# {
#     'activation': 'relu',
#     'batch_size': 32,
#     'learning_rate': 0.0034275039565641193,
#     'loss': 'mean_squared_error',
#     'momentum': 0.16182936755738428,
#     'optimizer': 'adam',
#     'output_activation': 'tanh',
#     'units': 512
# }

# inferencing

In [5]:
from mlflow.models import validate_serving_input

model_uri = 'runs:/ade621bfd87b4b17a168cc7861b29f88/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(X_test[:5])

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


array([[6.340846 ],
       [7.0023155],
       [6.7317724],
       [6.0285406],
       [6.765732 ]], dtype=float32)